# 1] You are working on a machine learning project where you have a dataset containing numerical and categorical features. You have identified that some of the features are highly correlated and there are missing values in some of the columns. You want to build a pipeline that automates the feature engineering process and handles the missing values.


## Design a pipeline that includes the following steps:
### Use an automated feature selection method to identify the important features in the dataset.
### Create a numerical pipeline that includes the following steps:
### Impute the missing values in the numerical columns using the mean of the column values.
### Scale the numerical columns using standardisation.
### Create a categorical pipeline that includes the following steps:
### Impute the missing values in the categorical columns using the most frequent value of the column.
### One-hot encode the categorical columns.
### Combine the numerical and categorical pipelines using a ColumnTransformer.
### Use a Random Forest Classifier to build the final model.
### Evaluate the accuracy of the model on the test dataset.
## Note: Your solution should include code snippets for each step of the pipeline, and a brief explanation of each step. You should also provide an interpretation of the results and suggest possible improvements for the pipeline.


In [1]:
import pandas as pd
import numpy as np


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.ensemble import RandomForestClassifier

In [3]:
df=pd.read_csv("train (2).csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.drop(columns=["PassengerId","Name","Cabin","Ticket"],axis=1,inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [6]:
df.shape

(891, 8)

In [7]:
# train_test_split
X_train,X_test,y_train,y_test=train_test_split(df.drop(["Survived"],axis=1),df["Survived"],test_size=0.2,random_state=29)

In [8]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
638,3,female,41.0,0,5,39.6875,S
36,3,male,NaN,0,0,7.2292,C
498,1,female,25.0,1,2,151.5500,S
828,3,male,NaN,0,0,7.7500,Q
590,3,male,35.0,0,0,7.1250,S


In [9]:
y_train.head()

638    0
36     1
498    0
828    1
590    0
Name: Survived, dtype: int64

In [10]:
# imputation transformer 
trf1=ColumnTransformer([
    ("impute_age",SimpleImputer(),[2]),
    ("impute_embarked",SimpleImputer(strategy="most_frequent"),[6])
],remainder="passthrough")

### => above code shows, 1st block of pipeline where we do imputation in column age with mean and in embarked with mode which is most frequent in column.

In [11]:
# one-hot encoding
trf2=ColumnTransformer([
    ("ohe_sex_embarked",OneHotEncoder(sparse=False,handle_unknown="ignore"),[1,6])
])

### => above code shows, 2nd block of pipeline where we do encoding with 'Onehotencoder()' in column age and embarked.

In [12]:
#scaling
trf3=ColumnTransformer([
    ("scaler",MinMaxScaler(),slice(0,10))
])

### => above code shows, 3rd block of pipeline where we do scaling with 'MinMaxScaler()' in all the column.

In [13]:
# feature selection
trf4=SelectKBest(score_func=chi2,k=5)

### => above code shows, 4th block of pipeline where we do selction of top 5 best feature with 'SelectKBest()'.

In [14]:
# model training
trf5=RandomForestClassifier()

### => finally we train the model using 'RandomForestClassifier()'

In [15]:
pipe=Pipeline([
    ("trf1",trf1),
    ("trf2",trf2),
    ("trf3",trf3),
    ("trf4",trf4),
    ("trf5",trf5)
])

In [16]:
from sklearn import set_config
set_config(display="diagram")

In [17]:
pipe.fit(X_train,y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scaler', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x7f38e1bfd240>)),
                ('trf5', RandomForestClassifier())])

### => above diagram shows the whole picture of pipeline

In [18]:
# from sklearn.pipeline import make_pipeline
# pipe=make_pipeline(trf1,trf2,trf3,trf4,trf5)
# pipe.fit(X_train,y_train)

### => we can also use 'make_pipeline()'.

In [19]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scaler', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=5, score_func=<function chi2 at 0x7f38e1bfd240>),
 'trf5': RandomForestClassifier()}

In [20]:
y_pred=pipe.predict(X_test)

In [21]:
from sklearn.metrics import classification_report,accuracy_score

In [22]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.70      0.89      0.78       116
           1       0.59      0.30      0.40        63

    accuracy                           0.68       179
   macro avg       0.65      0.59      0.59       179
weighted avg       0.66      0.68      0.65       179



In [23]:
accuracy_score(y_test,y_pred)

0.6815642458100558

# 2] Build a pipeline that includes a random forest classifier and a logistic regression classifier, and then use a voting classifier to combine their predictions. Train the pipeline on the iris dataset and evaluate its accuracy.

In [24]:
import seaborn as sns

In [25]:
data=sns.load_dataset("iris")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [26]:
data.shape

(150, 5)

In [27]:
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [28]:
data["species"].value_counts()

setosa        50
versicolor    50
virginica     50
Name: species, dtype: int64

In [29]:
# Step 1: Split the data into training and testing sets
X_train,X_test,y_train,y_test=train_test_split(data.drop(columns=["species"]),
                                               data["species"],
                                               test_size=0.2,
                                               random_state=129
                                              )

In [30]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier

In [31]:
# Step 2: Create individual classifiers with feature scaling
random_forest_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42))
])

logistic_regression_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=42))
])

In [32]:
# Step 3: Create the Voting Classifier
voting_classifier = VotingClassifier(estimators=[
    ('rf', random_forest_clf),
    ('lr', logistic_regression_clf)
], voting='soft')

In [33]:
# Step 4: Train the Voting Classifier on your data
voting_classifier.fit(X_train, y_train)


VotingClassifier(estimators=[('rf',
                              Pipeline(steps=[('scaler', StandardScaler()),
                                              ('rf',
                                               RandomForestClassifier(random_state=42))])),
                             ('lr',
                              Pipeline(steps=[('scaler', StandardScaler()),
                                              ('lr',
                                               LogisticRegression(multi_class='multinomial',
                                                                  random_state=42))]))],
                 voting='soft')

In [34]:
# Step 5: Make predictions using the Voting Classifier
predictions = voting_classifier.predict(X_test)

In [35]:
# Step 6: Evaluate the performance of the ensemble
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.9
